In [1]:
import os
from malaria_cnn import logger

In [2]:
os.chdir("../")
%pwd

'/home/fibonacci/projects/Detecting_Malaria_CNN'

In [3]:
from malaria_cnn.utils.common import read_yaml,create_directories
from malaria_cnn.constant import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from pathlib import Path

In [4]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [6]:
import os
import zipfile
from malaria_cnn import logger
from kaggle.api.kaggle_api_extended import KaggleApi
from malaria_cnn.utils.common import get_size

In [7]:
class DataIngestion:

    def __init__(self,config=DataIngestionConfig):
        self.config=config
        os.environ['KAGGLE_USERNAME'] = "KAGGLE_USERNAME"
        os.environ['KAGGLE_KEY'] = "KAGGLE_KEY"
        self.api = KaggleApi()
        self.api.authenticate()
    def download_file(self):
        try:
            dataset_id = "iarunava/cell-images-for-detecting-malaria"
            zip_download_dir = os.path.dirname(self.config.local_data_file)  # Extract directory path
            os.makedirs(zip_download_dir, exist_ok=True)  # Ensure directory exists
            logger.info(f"Downloading data from {dataset_id} into directory {zip_download_dir}")
            !kaggle datasets iarunava/cell-images-for-detecting-malaria -p {zip_download_dir}

            logger.info(f"Downloaded data from {dataset_id} into directory {zip_download_dir}")
        except Exception as e:
            logger.error(f"Error downloading data from {dataset_id} into directory {zip_download_dir}")
            raise e

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [8]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2024-04-27 01:54:08,464: yaml file:config/config.yaml load successfully: INFO
2024-04-27 01:54:08,636: yaml file:params.yaml load successfully: INFO
2024-04-27 01:54:08,640: Directory artifacts created successfully: INFO
2024-04-27 01:54:08,643: Directory artifacts/data_ingestion created successfully: INFO
2024-04-27 01:54:08,657: Downloading data from iarunava/cell-images-for-detecting-malaria into directory artifacts/data_ingestion: INFO


usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...
kaggle datasets: error: argument command: invalid choice: 'iarunava/cell-images-for-detecting-malaria' (choose from 'list', 'files', 'download', 'create', 'version', 'init', 'metadata', 'status')


2024-04-27 01:54:10,177: Downloaded data from iarunava/cell-images-for-detecting-malaria into directory artifacts/data_ingestion: INFO


IsADirectoryError: [Errno 21] Is a directory: 'artifacts/data_ingestion/cell-images-for-detecting-malaria.zip'